In [332]:
import pandas as pd
import numpy as np
import datetime as dt

In [333]:
csv_file = "All (1960-2023).csv"
usgs = pd.read_csv(csv_file, sep = ',', lineterminator='\n', dtype={'time':str})
usgs.head()

C:\Users\Vishal\AppData\Local\Temp\ipykernel_5872\3779433482.py:2: DtypeWarning:

Columns (1,2,3,4,6,7,8,9,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.



,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource\r
0,2023-10-12T21:41:21.480Z,36.4661674,-120.8755035,15.85,3.15,ml,60,135,0.1035,0.2,...,2023-10-13T19:19:02.194Z,"19 km WNW of New Idria, CA",earthquake,0.4,0.61,0.175,21,automatic,nc,nc\r
1,2023-10-08T21:30:23.900Z,38.8271667,-122.804,1.75,3.87,mw,108,20,0.006058,0.06,...,2023-10-14T02:06:32.597Z,"7 km NW of The Geysers, CA",earthquake,0.07,0.11,NaN,3,reviewed,nc,nc\r
2,2023-10-05T03:09:58.000Z,35.041,-117.661,0.79,3.52,ml,63,40,0.1102,0.15,...,2023-10-06T21:24:55.024Z,"5 km NNW of Boron, CA",earthquake,0.12,0.32,0.15,156,reviewed,ci,ci\r
3,2023-10-01T19:29:36.760Z,40.2915,-124.2905,9.59,3.61,mw,40,115,0.0308,0.17,...,2023-10-10T16:43:18.991Z,"4 km S of Petrolia, CA",earthquake,0.36,0.21,NaN,4,reviewed,nc,nc\r
4,2023-10-01T15:41:29.620Z,40.2951667,-124.287,9.8,4.09,mw,42,105,0.02685,0.17,...,2023-10-02T02:34:55.127Z,"3 km S of Petrolia, CA",earthquake,0.37,0.23,NaN,4,reviewed,nc,nc\r


In [334]:
#converting the Date column into datetime format
usgs["time"] = pd.to_datetime(usgs["time"], errors="coerce").dt.strftime("%Y-%m-%d")

In [335]:
usgs = usgs[(pd.to_datetime(usgs['time']) > pd.to_datetime('1960-01-01')) & (pd.to_datetime(usgs['time']) < pd.to_datetime('2023-01-01'))]

usgs['longitude'] = pd.to_numeric(usgs['longitude'], errors='coerce')
usgs['latitude'] = pd.to_numeric(usgs['latitude'], errors='coerce')
usgs['mag'] = pd.to_numeric(usgs['mag'], errors='coerce')

#filter the dataset by X > -123 and X < -113 and Y > 29 and Y < 39
usgs = usgs[usgs['longitude'] > -123]
usgs = usgs[usgs['longitude'] < -113]
usgs = usgs[usgs['latitude'] < 39]
usgs = usgs[usgs['latitude'] > 29]

usgs = usgs[usgs['mag'] >= 3.4]
usgs.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource\r
240,2022-12-31,33.397500,-116.393333,3.88,4.14,mw,132,16,0.07391,0.19,...,2023-09-22T21:50:30.029Z,"16 km N of Borrego Springs, CA",earthquake,0.1,0.38,NaN,6,reviewed,ci,ci\r
241,2022-12-31,34.355667,-116.921833,4.73,3.47,mw,121,25,0.07845,0.15,...,2023-03-07T19:00:01.040Z,"11km SSE of Lucerne Valley, CA",earthquake,0.09,0.41,NaN,4,reviewed,ci,ci\r
262,2022-12-17,37.918167,-122.304000,5.48,3.57,mw,170,19,0.01598,0.15,...,2023-07-27T08:15:34.318Z,"1km ENE of El Cerrito, CA",earthquake,0.1,0.17,NaN,4,reviewed,nc,nc\r
267,2022-12-07,35.508667,-118.391500,5.07,3.56,mw,58,21,0.1678,0.16,...,2023-02-11T23:03:13.040Z,"13km SE of Bodfish, CA",earthquake,0.13,0.9,NaN,4,reviewed,ci,ci\r
270,2022-12-06,35.935667,-120.012667,9.46,3.98,mw,66,75,0.08222,0.16,...,2023-02-11T23:03:09.040Z,"9km SSW of Kettleman City, CA",earthquake,0.26,0.54,NaN,5,reviewed,nc,nc\r


In [336]:
print(len(usgs))

11617


In [337]:
time = []
for i in usgs['time']:
    time.append(pd.to_datetime(i))
usgs['time'] = time

In [338]:
# usgs['time'] = usgs['time'].dt.to_timestamp()
usgs_grouped_counts = pd.DataFrame(usgs.groupby(usgs['time'].dt.to_period('D')).mag.count())
usgs_grouped_counts.rename(columns={'mag':'count'}, inplace=True)

usgs_grouped_max = pd.DataFrame(usgs.groupby(usgs['time'].dt.to_period('D')).mag.max())
print(usgs_grouped_counts)
print(usgs_grouped_max)

            count
time             
1960-01-02      1
1960-01-07      1
1960-01-11      1
1960-01-20      1
1960-01-26      1
...           ...
2022-12-05      1
2022-12-06      1
2022-12-07      1
2022-12-17      1
2022-12-31      2

[5997 rows x 1 columns]
             mag
time            
1960-01-02  4.04
1960-01-07  3.64
1960-01-11  3.79
1960-01-20  5.00
1960-01-26  4.90
...          ...
2022-12-05  3.59
2022-12-06  3.98
2022-12-07  3.56
2022-12-17  3.57
2022-12-31  4.14

[5997 rows x 1 columns]


In [339]:
import plotly.express as px
import plotly.graph_objects as go

In [340]:
usgs_grouped_counts.reset_index(inplace=True)
usgs_grouped_counts['time'] = usgs_grouped_counts['time'].dt.to_timestamp()

# Create a Plotly figure
fig = px.line(usgs_grouped_counts, x='time', 
              y='count', labels={'value': 'Magnitude'}, 
              title='USGS Earthquake Counts By Day')
fig.update_layout(
    width=1200,
    height=600,
)

fig.update_traces(line=dict(width=1.0))
# Add interactivity
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(dragmode='pan', hovermode='x')

# Show the interactive plot
fig.show()

In [341]:
usgs_grouped_max.reset_index(inplace=True)
usgs_grouped_max['time'] = usgs_grouped_max['time'].dt.to_timestamp()

# Create a Plotly figure
fig = px.line(usgs_grouped_max, x='time', 
              y='mag', labels={'value': 'Magnitude'}, 
              title='USGS Earthquake Max By Day')
fig.update_layout(
    width=1200,
    height=600,
)

fig.update_traces(line=dict(width=1.0))
# Add interactivity
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(dragmode='pan', hovermode='x')

# Show the interactive plot
fig.show()

July 6 - 316 earthquakes in a day. How is that possible?

In [342]:
#filter usgs_grouped_counts to the 10 largest values
usgs_grouped_counts_top10 = usgs_grouped_counts.nlargest(10, 'count')
usgs_grouped_counts_top10.head(10)

,time,count
5586,2019-07-06,316
3206,1992-06-28,123
865,1971-02-09,105
4004,1999-10-16,96
3433,1994-01-17,93
4881,2010-04-05,93
1869,1980-05-25,84
2611,1986-07-21,76
3207,1992-06-29,76
2243,1983-05-03,67


In [343]:
#filter the usgs_grouped_max to the 10 largest values 
usgs_grouped_max_top10 = usgs_grouped_max.nlargest(10, 'mag')
usgs_grouped_max_top10.head(10)

,time,mag
3206,1992-06-28,7.3
4880,2010-04-04,7.2
4004,1999-10-16,7.1
5586,2019-07-06,7.1
2939,1989-10-18,6.9
2242,1983-05-02,6.7
3433,1994-01-17,6.7
607,1968-04-09,6.6
865,1971-02-09,6.6
2753,1987-11-24,6.6


In [344]:
one_week = dt.timedelta(days=7)
usgs_grouped_counts_top10 = usgs_grouped_counts_top10.sort_values(by='time', ascending=True)

# Initialize an empty DataFrame to store the combined filtered data
filtered_data_df = pd.DataFrame()

# For each of the top 10 "mag" values, extract and combine the data points one week before and after
for index, row in usgs_grouped_counts_top10.iterrows():
    current_time = row['time']
    previous_time = current_time - one_week
    next_time = current_time + one_week
    
    filtered_data_before = usgs_grouped_counts[(usgs_grouped_counts['time'] >= previous_time) & (usgs_grouped_counts['time'] < current_time)].copy()
    filtered_data_before['days_until_large_value'] = (current_time - filtered_data_before['time']).dt.days  # Calculate days until the large value
    
    filtered_data_after = usgs_grouped_counts[(usgs_grouped_counts['time'] <= next_time) & (usgs_grouped_counts['time'] >= current_time)].copy()
    filtered_data_after['days_until_large_value'] = (current_time - filtered_data_after['time']).dt.days  # Calculate days until the large value
    
    filtered_data_df = pd.concat([filtered_data_df, filtered_data_before, filtered_data_after])

# Create a Plotly figure
fig = px.scatter(filtered_data_df, x='days_until_large_value', 
              y='count', 
              title='Top 10 Highest Earthquake Count Days, 1 Week Before/After',
              color='count',
              color_continuous_scale='viridis',
              labels={'count': 'Number of Earthquakes'},
)
fig.update_layout(
    width=1200,
    height=600,
)

# Add interactivity
fig.update_xaxes(title_text='Days Before/After Spike')
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(dragmode='pan', hovermode='x')

# Show the interactive plot
fig.show()

In [345]:
one_week = dt.timedelta(days=7)
usgs_grouped_max_top10 = usgs_grouped_counts_top10.sort_values(by='time', ascending=True)

# Initialize an empty DataFrame to store the combined filtered data
filtered_data_df = pd.DataFrame()

# For each of the top 10 "mag" values, extract and combine the data points one week before and after
for index, row in usgs_grouped_max_top10.iterrows():
    current_time = row['time']
    previous_time = current_time - one_week
    next_time = current_time + one_week
    
    filtered_data_before = usgs_grouped_max[(usgs_grouped_max['time'] >= previous_time) & (usgs_grouped_max['time'] < current_time)].copy()
    filtered_data_before['days_until_large_value'] = (current_time - filtered_data_before['time']).dt.days  # Calculate days until the large value
    
    filtered_data_after = usgs_grouped_max[(usgs_grouped_max['time'] <= next_time) & (usgs_grouped_max['time'] >= current_time)].copy()
    filtered_data_after['days_until_large_value'] = (current_time - filtered_data_after['time']).dt.days  # Calculate days until the large value
    
    filtered_data_df = pd.concat([filtered_data_df, filtered_data_before, filtered_data_after])

# Create a Plotly figure
fig = px.scatter(filtered_data_df, x='days_until_large_value', 
              y='mag', 
              title='Top 10 Largest Earthquake Days, 1 Week Before/After',
              color='mag',
              color_continuous_scale='viridis',
              labels={'count': 'Magnitude'},
)
fig.update_layout(
    width=1200,
    height=600,
)

# Add interactivity
fig.update_xaxes(title_text='Days Before/After Spike')
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(dragmode='pan', hovermode='x')

# Show the interactive plot
fig.show()